In [1]:
# Paso 2.
# Obtiene la variable DNI a partir de GHI usando el modelo DISC de NREL.

import numpy as np
import pandas as pd

import xarray as xr

# Inicializamos el dashboard de cómputo distribuido.
#from dask.distributed import Client
#c_lat = 1
#c_lon = 1
#client = Client( n_workers = 5, threads_per_worker = 10,
#    memory_limit = "250MB" )
#client

KeyboardInterrupt: 

In [2]:
# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

# Modelo NREL-DISC, calcula DNI usando solo GHI.
def DISC(ds):
    # Pasamos a Dataframe
    df = ds.to_dataframe().reset_index().set_index("time")
    df = pd.concat( [df, df_t], axis = 1 )

    # Latitud y longitud de la celda.
    lat = df["lat"].iloc[0]
    lon = df["lon"].iloc[0]

    # Ángulo horario.
    df["Hour Angle"] = ( 15 * ( df.index.hour - 12
        - 0.5 + df["EQT"]/60 + ((lon-TZ*15)*4)/60 ) )
    # Posiciones del analema solar.
    df["Sx"] = cos(df["DEC"])*cos(df["lon_subs"]-lon)
    df["Sy"] = ( cos(lat)*sin(df["DEC"])
        - sin(lat)*cos(df["DEC"])*cos(df["lon_subs"]-lon) )
    df["Sz"] = ( sin(lat)*sin(df["DEC"])
        - cos(lat)*cos(df["DEC"])*cos(df["lon_subs"]-lon) )
    # Ángulo del cénit solar.
    df["Zenith"] = acos(df["Sz"])
    # Masa de aire.
    df["AM"] = ( ( 1/cos(df["Zenith"])
        + 0.15/(93.885 - df["Zenith"])**1.253 )
        * df["Pressure"]/1013.25 )
    df["AM"] = df["AM"].where( df["Zenith"] < 85.5, 0 )
    df["Kt"] = ( df["GHI"] / (cos(df["Zenith"])*df["ETR"]) )
    df["Kt"] = df["Kt"].where( df["AM"] > 0, 0 )
    df["A_1"] = ( -5.743 + 21.77*df["Kt"]
        - 27.49*df["Kt"]**2 + 11.56*df["Kt"]**3 )
    df["A_2"] = ( 0.512 - 1.56*df["Kt"]
        + 2.286*df["Kt"]**2 - 2.222*df["Kt"]**3 )
    df["A"] = ( df["A_1"].where( df["Kt"] > 0.6, df["A_2"]
            ).where( df["Kt"] > 0, 0 ) )
    df["B_1"] = ( 41.4 - 118.5*df["Kt"]
        + 66.05*df["Kt"]**2 + 31.9*df["Kt"]**3 )
    df["B_2"] = 0.37 + 0.962*df["Kt"]
    df["B"] = df["B_1"].where( df["Kt"] > 0.6, df["B_2"]
        ).where( df["Kt"] > 0, 0 )
    df["C_1"] = ( -47.01 + 184.2*df["Kt"]
        - 222*df["Kt"]**2 + 73.81*df["Kt"]**3 )
    df["C_2"] = -0.28 + 0.932*df["Kt"] - 2.048*df["Kt"]**2
    df["C"] = df["C_1"].where( df["Kt"] > 0.6, df["C_2"]
        ).where( df["Kt"] > 0, 0 )
    df["D_Kn"] = df["A"] + df["B"] * np.exp( df["C"] * df["AM"] )
    df["D_Kn"] = df["D_Kn"].where( df["Kt"] > 0, 0 )
    df["Knc"] = ( 0.886 - 0.122*df["AM"] + 0.0121*df["AM"]**2
        - 0.000653*df["AM"]**3 + 0.000014*df["AM"]**4 )
    df["Knc"] = df["Knc"].where( df["Kt"] > 0, 0 )
    # Radiación normal directa.
    df["DNI"] = df["ETR"] * ( df["Knc"] - df["D_Kn"] )
    df["DNI"] = df["DNI"].where( df["Kt"] > 0, 0
        ).where( df["DNI"] > 0, 0 ).round( decimals = 2 )
    # Quitamos todas las columnas de apoyo
    df = df.drop( [ "ETR", "DEC", "EQT",
    "Hour Angle", "lon_subs", "Sx", "Sy", "Sz", "Zenith",
    "AM", "Kt", "A_1", "A_2", "A", "B_1", "B_2", "B",
    "C_1", "C_2", "C", "D_Kn", "Knc" ], axis = 1 )

    # Convertimos a Dataset.
    return df.reset_index().set_index( ["time", "lat", "lon"]
        ).astype( float ).to_xarray()

In [3]:
def DISC(ds):
    #df = ds.to_dataframe()
    #df = df*2
    #return df.to_xarray()
    return ds*2

In [4]:
# Cargamos el archivo.
n = 0
path_1 = f"../temp/WRF/WRF_miroc_1985_2014_{n}.nc"
path_2 = f"../temp/WRF/WRF_miroc_1985_2014_{n}_DNI.nc"
ds = xr.open_dataset(path_1)

# Corremos el modelo disc con cómputo distribuido.
ds_2 = xr.map_blocks(DISC, ds)

# Guardamos el archivo.
ds.to_netcdf(path_2)

In [3]:
# Cargamos el archivo.
n = 0
path_1 = f"../temp/WRF/WRF_miroc_1985_2014_{n}.nc"
path_2 = f"../temp/WRF/WRF_miroc_1985_2014_{n}_DNI.nc"
ds = xr.open_dataset(path_1)

# Huso horario.
TZ = -6

df_t = ds["time"].to_dataframe()
df_t["Day Angle"] = 6.283185 * ( df_t.index.dayofyear - 1 ) / 365
df_t["ETR"] = ( 1370 * ( 1.00011 + 0.034221*np.cos(df_t["Day Angle"])
    + 0.00128*np.sin(df_t["Day Angle"])
    + 0.000719*np.cos(2*df_t["Day Angle"])
    + 0.000077*np.sin(2*df_t["Day Angle"]) ) )
# Declinación
df_t["DEC"] = ( ( 0.006918 - 0.399912 * np.cos(df_t["Day Angle"])
    + 0.070257*np.sin(df_t["Day Angle"])
    - 0.006758*np.cos(2*df_t["Day Angle"])
    + 0.000907*np.sin(2*df_t["Day Angle"])
    - 0.002697*np.cos(3*df_t["Day Angle"])
    + 0.00148*np.sin(3*df_t["Day Angle"]) ) * 180/np.pi )
# Ecuación del tiempo.
df_t["EQT"] = ( ( 0.000075 + 0.001868*np.cos(df_t["Day Angle"])
    - 0.032077*np.sin(df_t["Day Angle"])
    - 0.014615*np.cos(2*df_t["Day Angle"])
    - 0.040849*np.sin(2*df_t["Day Angle"])) * 229.18 )
# Longitud del punto subsolar.
df_t["lon_subs"] = -15 * ( df_t.index.hour - TZ + df_t["EQT"]/60 )

df_t = df_t.drop( ["time", "Day Angle"], axis = 1 )

# Corremos el modelo disc con cómputo distribuido.
ds_2 = xr.map_blocks(DISC, ds)

# Guardamos el archivo.
ds.to_netcdf(path_2)